<h2><i>kiara</i>: Network Analysis</h2>

Welcome back! Now that we're comfortable with what <i>kiara</i> looks like and what it can do to help track your data and your research process, let's try out some of the digital analysis tools, starting with <b>Network Analysis</b>.

<h2>Why Network Analysis?</h2>

Network Analysis offers a computational and quantitative means to examine and explore relational objects, with proxies to measure structural roles and concepts such as power and influence. Doing so digitally - and at scale - also allows us to consider these kinds of questions with large amounts of material or documents that was not  heretofore manageable with qualitative or manual approaches.

>INSERT REFERENCES OR WIDER REFERENCING HERE: PROGRAMMING HISTORIAN LINKS W/ BIBLIOGRAPHY SECTION BELOW

<h3>Getting Started</h3>

Let's start by double checking that we have all the required plugins and setting up an API for us to use <i>kiara</i>. We'll do this all in one go this time, but if you're unsure, feel free to head back to the <a href="http://dharpa.org/kiara.documentation/latest/workshop/workshop/">installation notebook</a> to look over this section again.

In [1]:
import csv
import networkx as nx
import matplotlib.pyplot as plt

try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()

from kiara.api import KiaraAPI
kiara = KiaraAPI.instance()

Great, we're all set up. We're going to import some data again like in the first notebook, but this time we're going to use a local file, using the kiara function `import.local.file`. We're using sample data again here, but you can also use this function to import your own data in the future. 

The data we're using here is a sample taken from the <b>correspSearch</b> dataset collated by the Berlin-Brandenburg Academy of Sciences and available on the 'LetterSampo' portal created by the Reassembling the Republic of Letters project team. For more on these projects see <a href="https://seco.cs.aalto.fi/projects/rrl/">here</a>.

A continually growing 'crowd-sourced' dataset, the correspSearch collection features letters from German history across a broad range of history, contributed by various individuals and larger research groups. Using quantitative network analysis on this dataset might offer insights into the most prolific writer in the dataset so-far, which actor connected the most people, or who operated in closely knit writing groups. Although we can also use network analysis to explore or 'map' our datasets if we don't know much about them, in this notebook the research questions and module parameters have been built around and defined by the information we already have about the data. It's important that we acknowledge this now as a core factor in the decisions already made for this process, but we'll also return to this throughout the notebook.

Let's use the <i>kiara</i> function `import.local.file` then to access our datset, specifying the path to the csv file in our <span style="color:green">inputs</span> and saving the <span style="color:red">outputs</span> of the function as '<b>correspsearch</b>'.

In [2]:
correspsearch = kiara.run_job('import.local.file', inputs={'path':'CKCC.csv'})
correspsearch

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────                                                      │
│   file    Source,Target                                                                                                                  │
│           "Charas, Moyse, 1619-1698","Huygens, Christiaan, 1629-1695"                                                                    │
│           "Hartzwich, Peter, fl. 1632","Groot, Hugo de, 1583-1645"                                                                       │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Huygens, Constantijn, 1596-1687"                                                                │
│           "Heinsius, Daniel, 1580-1655","Huygens, Constantijn, 1596-1687"                                                                │
│           "Heinsius, Nicolaas, 1620-1681","Huygens, Christiaan, 1629-1695"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Boulliau, Ismaël, 1605-1694"                                                                  │
│           "Heinsius, Nicolaas, 1620-1681","Boulliau, Ismaël, 1605-1694"                                                                  │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Gronovi

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────                                                      │
│   file    Source,Target                                                                                                                  │
│           "Charas, Moyse, 1619-1698","Huygens, Christiaan, 1629-1695"                                                                    │
│           "Hartzwich, Peter, fl. 1632","Groot, Hugo de, 1583-1645"                                                                       │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Groot, Hugo de, 1583-1645"                                                                      │
│           "Heinsius, Daniel, 1580-1655","Huygens, Constantijn, 1596-1687"                                                                │
│           "Heinsius, Daniel, 1580-1655","Huygens, Constantijn, 1596-1687"                                                                │
│           "Heinsius, Nicolaas, 1620-1681","Huygens, Christiaan, 1629-1695"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Boulliau, Ismaël, 1605-1694"                                                                  │
│           "Heinsius, Nicolaas, 1620-1681","Boulliau, Ismaël, 1605-1694"                                                                  │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Dati, Carlo Roberto, 1619-1676"                                                               │
│           "Heinsius, Nicolaas, 1620-1681","Gronovi

<h2>Creating a Network</h2>

Time to make our network from this data. Let's start by searching for the <i>kiara</i> modules that are included in the `kiara_plugin.network_analysis` package.

In [3]:
infos = metadata = kiara.retrieve_operations_info()
operations = {}
for op_id, info in infos.item_infos.items():
    if info.context.labels.get("package", None) == "kiara_plugin.network_analysis":
        operations[op_id] = info

print(operations.keys())

dict_keys(['create.network_data.from.tables', 'export.network_data.as.csv_files', 'export.network_data.as.graphml_file', 'export.network_data.as.sql_dump', 'export.network_data.as.sqlite_db', 'network_data.check_clusters'])
dict_keys(['create.network_data.from.tables', 'export.network_data.as.csv_files', 'export.network_data.as.graphml_file', 'export.network_data.as.sql_dump', 'export.network_data.as.sqlite_db', 'network_data.check_clusters'])


There's lots of options for analysis, but we want to make our network first. Let's have a look what we need with the function `create.network_data.from.tables` using `kiara.retrieve_operation_info` once more.

<span style="color:blue">this will make more sense/have more options when analysis modules are incorporated into the network plugin (lol)</span>

In [4]:
kiara.retrieve_operation_info('create.network_data.from.tables')

Author(s)                                                                                                                                  
                     Lena Jaskov         helena.jaskov@uni.lu                                                                               
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         network_analysis                                                                                          
                     Labels       package: kiara_plugin.network_analysis                                                                    
                     References   source_repo: ]8;id=993839;https://github.com/DHARPA-Project/kiara_plugin.network_analysis\https://github.com/DHARPA-Project/kiara_plugin.network_analysis]8;;\                              
                                  documentation: ]8;id=422467;https://DHARPA-Project.github.io/kiara_plugin.network_analysis/\https://DHARPA-Project.github.io/kiara_plugin.network_analysis/]8;;\                            
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a graph object from one or two tables.                                                          
                                                                                                                                            
                                     This module needs at least one table as input, providing the edges of the resulting network data       
                                     set.                                                                                                   
                                     If no further table is created, basic node information will be automatically created by using unique   
                                     values from                                                                                            
                                     the edges source and target columns.                                                                   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name           type     description                            Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       edges                table    A table that contains the edges        yes        -- no default --     
                                                                     data.                                                                  
                                       source_column_name   string   The name of the source column name     no         source               
                                                                     in the edges table.                                                    
                                       target_column_name   string   The name of the target column name     no         target               
                      

Author(s)                                                                                                                                  
                     Lena Jaskov         helena.jaskov@uni.lu                                                                               
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         network_analysis                                                                                          
                     Labels       package: kiara_plugin.network_analysis                                                                    
                     References   source_repo: ]8;id=993839;https://github.com/DHARPA-Project/kiara_plugin.network_analysis\https://github.com/DHARPA-Project/kiara_plugin.network_analysis]8;;\                              
                                  documentation: ]8;id=422467;https://DHARPA-Project.github.io/kiara_plugin.network_analysis/\https://DHARPA-Project.github.io/kiara_plugin.network_analysis/]8;;\                            
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a graph object from one or two tables.                                                          
                                                                                                                                            
                                     This module needs at least one table as input, providing the edges of the resulting network data       
                                     set.                                                                                                   
                                     If no further table is created, basic node information will be automatically created by using unique   
                                     values from                                                                                            
                                     the edges source and target columns.                                                                   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name           type     description                            Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       edges                table    A table that contains the edges        yes        -- no default --     
                                                                     data.                                                                  
                                       source_column_name   string   The name of the source column name     no         source               
                                                                     in the edges table.                                                    
                                       target_column_name   string   The name of the target column name     no         target               
                      

Like other network analysis tools, <i>kiara</i> first needs the data as an edge table. This means we first have to transform the csv file we imported earlier into a table before we can create the network data. Let's start by using the `create.table.from.file` function that we used in the first notebook and storing this as our <b>edges</b>, then use this to create our network data using the `create.network_data.from.tables` that we just read about. In this, we are defining two different sets of <span style="color:green">inputs</span>, overriding the first variable once we have used it to create our table.

If we want, we can also import a separate table with the nodes in, but this is optional, and for the moment let's stick with just the edge table. We'll store this again at the end in the variable <b>correspsearch</b> for us to use again in a bit.

<span style="color:blue">also include information on the f0/f1 problem, if this is not fixed by then - we'll also need to write in dropping the first row (i.e. 'Source' and 'Target') if this is the case. Otherwise, write instructions for identifying source/target columns, alongside more information on the rest of the possible inputs (so that it makes sense for the lesmis info later)</span>

In [5]:
inputs = {
    "file": correspsearch['file']
}

outputs = kiara.run_job('create.table.from.file', inputs={'file': correspsearch['file']})

edges = outputs['table']

inputs = {
    'edges': edges,
    'source_column_name': 'f0',
    'target_column_name': 'f1'
}

correspsearch = kiara.run_job('create.network_data.from.tables', inputs=inputs)
correspsearch

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│   network_data                                                                                                                           │
│                  Table: edges                                                                                                            │
│                                                                                                                                          │
│                    source                                              target                                                            │
│                   ───────────────────────────────────────────────────────────────────────────────────────────────────                    │
│                    Source                                              Target                                                            │
│                    Charas, Moyse, 1619-1698                            Huygens, Christiaan, 1629-1695                                    │
│                    Hartzwich, Peter, fl. 1632                          Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Huygens, Constantijn, 1596-1687                                   │
│                    Heinsius, Daniel, 1580-1655                         Huygens, Constantijn, 1596-1687                                   │
│                    Heinsius, Nicolaas, 1620-1681                       Huygens, Christiaan, 1629-1695                                    │
│                    ...                                                 ...                                                               │
│                    ...                                                 ...                                                               │
│                    Zeeland, States of                                  Groot, Hugo de, 1583-1645                                         │
│                    Zeeland, States of             

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│   network_data                                                                                                                           │
│                  Table: edges                                                                                                            │
│                                                                                                                                          │
│                    source                                              target                                                            │
│                   ───────────────────────────────────────────────────────────────────────────────────────────────────                    │
│                    Source                                              Target                                                            │
│                    Charas, Moyse, 1619-1698                            Huygens, Christiaan, 1629-1695                                    │
│                    Hartzwich, Peter, fl. 1632                          Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Groot, Hugo de, 1583-1645                                         │
│                    Heinsius, Daniel, 1580-1655                         Huygens, Constantijn, 1596-1687                                   │
│                    Heinsius, Daniel, 1580-1655                         Huygens, Constantijn, 1596-1687                                   │
│                    Heinsius, Nicolaas, 1620-1681                       Huygens, Christiaan, 1629-1695                                    │
│                    ...                                                 ...                                                               │
│                    ...                                                 ...                                                               │
│                    Zeeland, States of                                  Groot, Hugo de, 1583-1645                                         │
│                    Zeeland, States of             

Great - this has made a <i>kiara</i> network data object, and the output is showing the edge table and node table for the network. As we didn't give it a node table to start with this, it has extracted the information for the nodes from the edges instead.

As we can see, some of the edges are listed more than once, where more than one letter was written from one person to another. There's obviously more information about the network than just a list of edges and nodes then - let's find out some more information about our network object then.

<h2>Network Data</h2>

Let's start by having a look at the information for our network using the `get_property_data` function. As we're querying the `network_data` part of our network object, we'll save this as <b>correspsearch</b> for the moment.

In [6]:
correspsearch = correspsearch['network_data']
correspsearch.get_property_data('metadata.graph_properties')

 number_of_nodes            1928                                 
  properties_by_graph_type   {                                    
                               "graph_type": "directed",          
                               "number_of_edges": 2943            
                             }                                    
                             {                                    
                               "graph_type": "undirected",        
                               "number_of_edges": 2333            
                             }                                    
                             {                                    
                               "graph_type": "directed-multi",    
                               "number_of_edges": 20030           
                             }                                    
                             {                                    
                               "graph_type": "undirected-multi",  
                               "number_of_edges": 20030           
                             }                                    
  number_of_self_loops       92                                   
  number_of_parallel_edges   17087

 number_of_nodes            1928                                 
  properties_by_graph_type   {                                    
                               "graph_type": "directed",          
                               "number_of_edges": 2943            
                             }                                    
                             {                                    
                               "graph_type": "undirected",        
                               "number_of_edges": 2333            
                             }                                    
                             {                                    
                               "graph_type": "directed-multi",    
                               "number_of_edges": 20030           
                             }                                    
                             {                                    
                               "graph_type": "undirected-multi",  
                               "number_of_edges": 20030           
                             }                                    
  number_of_self_loops       92                                   
  number_of_parallel_edges   17087

Doing this gives us the total number of nodes, but also gives us an idea of the different kind of graphs we might chose to use for this dataset - <b>Directed</b>, <b>Undirected</b>, <b>Multi-Directed</b>, and <b>Multi-Undirected</b>. Having this kind of information accessible means we can make more informed decisions about the next steps that might work with our research or digital analysis, especially those that are sometimes automated for us.

Let's get some more information about the network as a whole then, using the `network_data.extract_largest_component` function. This works out how many different distinct components there are in a network, and also gives us the largest component on its own. We'll have a quick look at how it works first.

In [ ]:
kiara.retrieve_operation_info('network_data.extract_largest_component')

In [ ]:
output = kiara.run_job('network_data.extract_largest_component', inputs={'network_data':correspsearch})
output

For now, let's save our largest component in the variable `network_data` for later - we'll use this for the rest of our experiments rather than the full network, and make sure we're tracing this using <i>kiara</i>.

Let's have a look at the information for this largest component, using our `get_property_data` function again.

In [7]:
#network_data = output['largest_component']

#network_data.get_property_data('metadata.graph_properties')

network_data = correspsearch

<h2>Onboarding Data: An Alternative</h2>

So far then, we have created a network object in <i>kiara</i> by importing a csv from a local path.

But what about other formats? Let's pause quickly, and have a look at importing a <b>gml</b> file instead. 

Here we will use a different sample dataset, <a href="http://www-personal.umich.edu/~mejn/netdata/">co-appearance network</a> of characters in Victor Hugo's novel <i>Les Miserables</i>, already in gml format. 

Let's have a look at the function `onboard.gml_file` and how this will work for us.

In [ ]:
kiara.retrieve_operation_info('onboard.gml_file')

We need a local file path again, and we can go ahead and save this as <b>lesmis</b>.

In [ ]:
lesmis = kiara.run_job('onboard.gml_file', inputs={'path':'lesmis.gml'})
lesmis

As we can see, this module not only imports the gml file into <i>kiara</i> but automatically converts it into a <i>kiara</i> network object for us. Great!

Here we can see that the edge table has a 'value' column to indicate edges weights that has also been automatically included with the gml data.

We'll leave this <i>Les Miserables</i> network for now, but it's useful to see this other option for importing data for networks. If you want to experiment with this dataset later, feel free to come back to it!

<h2>Network Analysis: Statistical Measures</h2>

Ok, let's head back to our correspSearch largest component dataset, stored in the variable <b>network_data</b>. We've already had a look at some graph wide measures, so let's start looking at some node specific measurements.

Let's start with degree, using `create.degree_rank_list`. This module allows us to calculate degree as both <b>undirected</b> and <b>weighted</b>. In this epistolary network, <b>undirected degree</b> counts the number of individual correspondents each person has, whereas <b>weighted degree</b> counts the total number of incoming and outgoing letters for each actor in the network. 

Let's use our `retrieve_operation_info` function to have a look at what we need to calculate these degrees.

In [ ]:
kiara.retrieve_operation_info('create.degree_rank_list')

So we've already computed the largest component to use as the `network_data` input, and we want to calculate the weighted degree meaures, so we'll leave the default as 'True'. Unlike the <i>Les Miserables</i> network, we don't have a pre-existing weight value for the edges, but we do know that there are parallell edges from multiple letters between correspondents, so we'll allow the module to aggregate the edges and set this as a weight. 

In creating this module, assumptions have already been made that we are working with both a single node type and a single edge type network. Again, a lot of the parameters have been set based on what we already know about the dataset, but we also need to acknowledge this as an active decision that has been 'pre-made' as part of the research process.

The inputs for `create.degree_rank_list` are prompting us to reflect on the decisions we are making as we are going along, and think about how our data fits into these kind of measurements, but by doing it in <i>kiara</i>, these inputs also allows us to <i>track</i> these decisions, as we will see more of later.

Let's give it a go then.

In [8]:
output = kiara.run_job('create.degree_rank_list', inputs={'network_data':network_data})
output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

In [ ]:
kiara.retrieve_operation_info('create.betweenness_rank_list')

In [9]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.betweenness_rank_list', inputs={'network_data':centrality_network})

output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

In [ ]:
kiara.retrieve_operation_info('create.eigenvector_rank_list')

In [10]:
centrality_network = output['centrality_network']

output = kiara.run_job('create.eigenvector_rank_list', inputs={'network_data':centrality_network})

output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

In [ ]:
kiara.retrieve_operation_info('compute.modularity_group')

In [11]:
network = output['centrality_network']

output = kiara.run_job('compute.modularity_group', inputs={'network_data':network})

output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   maximum_modularity   17                                                                                                                │
│   modularity_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641 

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   maximum_modularity   17                                                                                                                │
│   modularity_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641 

In [ ]:
kiara.retrieve_operation_info('create.cut_point_list')

In [13]:
centrality_network = output['modularity_network']

output = kiara.run_job('create.cut_point_list', inputs={'network_data':centrality_network})

output

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                value                                                                                                             │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   centrality_network                                                                                                                     │
│                        Table: edges                                                                                                      │
│                                                                                                                                          │
│                          source                                              target                                         weight       │
│                         ───────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Huygens, Constantijn, 1596-1687                3            │
│                          Elisabeth of the Palatinate (Princess), 1618-1680   Descartes, René, 1596-1650                     33           │
│                          Finet, John (Sir), 1571-1641                        Huygens, Constantijn, 1596-1687                1            │
│                          Nassau, Frederik van, 1624-1672                     Huygens, Constantijn, 1596-1687                4            │
│                          Ruppa                                               Huygens, Constantijn, 1596-1687                1            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Constantijn, 1596-1687                3            │
│                          Menjot, Antoine, 1615-1696                          Huygens, Christiaan, 1629-1695                 1            │
│                          Hartaing, Olympe, fl. 1603-1635                     Huygens, Constantijn, 1596-1687                1            │
│                          Aerssen, François van, 1572-1641                    Huygens, Constantijn, 1596-1687                6            │
│                          Noot, Louise van der, fl. 1627-1632                 Huygens, Constantijn, 1596-1687                2            │
│                          Vos van Steenwijck, fl. 1634-1635                   Huygens, Constantijn, 1596-1687                6            │
│                          Lesno, van, fl. 1633                                Huygens, Constantijn, 1596-1687                1            │
│                          Sandilands, James, fl. 1637-1638                    Huygens, Constantijn, 1596-1687                1            │
│                          Diodati, Elia, 1576-1661                            Huygens, Constantijn, 1596-1687                5            │
│                          Calandrin, Caesar, 1595-1665                        Huygens, Constantijn, 1596-1687                22           │
│                          Sidney, Robert, 1595-1677                           Huygens, Constantijn, 1596-1687                1            │
│                          ...                                                 ...                                            ...          │
│                          ...                                                 ...                                            ...          │
│                          Banér, Johann, 1596-1641                            Schörlings, Balthasar, d.1638                  1            │
│                          Horn, Gustav Karlsson af 

In [14]:
output['centrality_network'].lineage

create.cut_point_list
└── input: network_data (network_data) = 5ac68e91-2eb8-45bb-9f54-d3e4ea2cd7a7
    └── compute.modularity_group
        ├── input: network_data (network_data) = 38e13cac-3429-4f16-8d25-4c1ef90b64ab
        │   └── create.eigenvector_rank_list
        │       ├── input: iterations (integer) = eb09a96d-8720-49d7-9296-d6c3d4b2d855
        │       ├── input: network_data (network_data) = 4c7aa735-2c3a-4f7c-8f25-ff83af80aea2
        │       │   └── create.betweenness_rank_list
        │       │       ├── input: network_data (network_data) = 18d54ac0-e59c-4475-a004-2a523f0c6b4d
        │       │       │   └── create.degree_rank_list
        │       │       │       ├── input: network_data (network_data) = d0f3593f-57a9-425f-97f2-4aa7c66c331d
        │       │       │       │   └── create.network_data.from.tables
        │       │       │       │       ├── input: edges (table) = acc7fea1-a1f4-49cb-bd42-c529c8e268f8
        │       │       │       │       │   └── create.table
        │       │       │       │       │       └── input: file (file) = ec6b5313-8313-4b01-8186-aad8bdf6fcb5
        │       │       │       │       │           └── import.local.file
        │       │       │       │       │               └── input: path (string) = c82447e5-557c-4ffe-8ad9-639f22dde87e
        │       │       │       │       ├── input: edges_column_map (dict) = d62aa891-89a0-4731-8a8d-2f7f6e80985e
        │       │       │       │       ├── input: id_column_name (string) = 09067550-78bb-4ff3-8fb3-79cb1af92dc2
        │       │       │       │       ├── input: label_column_name (string) = 97bb1d77-bf78-4fc1-9a97-2d924a7b436e
        │       │       │       │       ├── input: nodes (table) = d98ae9f6-a75f-4ab5-8eff-d6d1714b7b37
        │       │       │       │       ├── input: nodes_column_map (dict) = e2da9652-b256-4b64-ac61-006b58590613
        │       │       │       │       ├── input: source_column_name (string) = c2374399-19f0-4c41-b3ca-4e468ac64151
        │       │       │       │       └── input: target_column_name (string) = 7fdcb041-dd08-4b34-909a-1fcfa444b581
        │       │       │       ├── input: weight_column_name (string) = 080bb956-fa09-4ecd-a25d-754fbd526cb5
        │       │       │       └── input: weighted_degree (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       │       ├── input: weight_column_name (string) = 080bb956-fa09-4ecd-a25d-754fbd526cb5
        │       │       ├── input: weight_meaning (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       │       └── input: weighted_betweenness (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       ├── input: weight_column_name (string) = 080bb956-fa09-4ecd-a25d-754fbd526cb5
        │       ├── input: weight_meaning (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       └── input: weighted_eigenvector (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        └── input: number_of_communities (integer) = 4449f269-a388-45b1-9b87-88a24b53fe00

create.cut_point_list
└── input: network_data (network_data) = 5ac68e91-2eb8-45bb-9f54-d3e4ea2cd7a7
    └── compute.modularity_group
        ├── input: network_data (network_data) = 38e13cac-3429-4f16-8d25-4c1ef90b64ab
        │   └── create.eigenvector_rank_list
        │       ├── input: iterations (integer) = eb09a96d-8720-49d7-9296-d6c3d4b2d855
        │       ├── input: network_data (network_data) = 4c7aa735-2c3a-4f7c-8f25-ff83af80aea2
        │       │   └── create.betweenness_rank_list
        │       │       ├── input: network_data (network_data) = 18d54ac0-e59c-4475-a004-2a523f0c6b4d
        │       │       │   └── create.degree_rank_list
        │       │       │       ├── input: network_data (network_data) = d0f3593f-57a9-425f-97f2-4aa7c66c331d
        │       │       │       │   └── create.network_data.from.tables
        │       │       │       │       ├── input: edges (table) = acc7fea1-a1f4-49cb-bd42-c529c8e268f8
        │       │       │       │       │   └── create.table
        │       │       │       │       │       └── input: file (file) = ec6b5313-8313-4b01-8186-aad8bdf6fcb5
        │       │       │       │       │           └── import.local.file
        │       │       │       │       │               └── input: path (string) = c82447e5-557c-4ffe-8ad9-639f22dde87e
        │       │       │       │       ├── input: edges_column_map (dict) = d62aa891-89a0-4731-8a8d-2f7f6e80985e
        │       │       │       │       ├── input: id_column_name (string) = 09067550-78bb-4ff3-8fb3-79cb1af92dc2
        │       │       │       │       ├── input: label_column_name (string) = 97bb1d77-bf78-4fc1-9a97-2d924a7b436e
        │       │       │       │       ├── input: nodes (table) = d98ae9f6-a75f-4ab5-8eff-d6d1714b7b37
        │       │       │       │       ├── input: nodes_column_map (dict) = e2da9652-b256-4b64-ac61-006b58590613
        │       │       │       │       ├── input: source_column_name (string) = c2374399-19f0-4c41-b3ca-4e468ac64151
        │       │       │       │       └── input: target_column_name (string) = 7fdcb041-dd08-4b34-909a-1fcfa444b581
        │       │       │       ├── input: weight_column_name (string) = 080bb956-fa09-4ecd-a25d-754fbd526cb5
        │       │       │       └── input: weighted_degree (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       │       ├── input: weight_column_name (string) = 080bb956-fa09-4ecd-a25d-754fbd526cb5
        │       │       ├── input: weight_meaning (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       │       └── input: weighted_betweenness (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       ├── input: weight_column_name (string) = 080bb956-fa09-4ecd-a25d-754fbd526cb5
        │       ├── input: weight_meaning (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        │       └── input: weighted_eigenvector (boolean) = 3447f665-9bbf-469e-a4d5-1f15964adff0
        └── input: number_of_communities (integer) = 4449f269-a388-45b1-9b87-88a24b53fe00

In [15]:
kiara.retrieve_operation_info('export.network_data.as.graphml_file')

Author(s)                                                                                                                                  
                     Lena Jaskov         helena.jaskov@uni.lu                                                                               
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         network_analysis                                                                                          
                     Labels       package: kiara_plugin.network_analysis                                                                    
                     References   source_repo: ]8;id=813188;https://github.com/DHARPA-Project/kiara_plugin.network_analysis\https://github.com/DHARPA-Project/kiara_plugin.network_analysis]8;;\                              
                                  documentation: ]8;id=110383;https://DHARPA-Project.github.io/kiara_plugin.network_analysis/\https://DHARPA-Project.github.io/kiara_plugin.network_analysis/]8;;\                            
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Export network data as graphml file.                                                                   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name        type           description                         Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       network_data      network_data   A value of type 'network_data'.     yes        -- no default --     
                                       base_path         string         The directory to export the         no         -- no default --     
                                                                        file(s) to.                                                         
                                       name              string         The (base) name of the exported     no         -- no default --     
                                                                        file(s).                                                            
                                       export_metadata   boolean        Whether to also export the value    no         False                
                                                                        metadata.                                                           
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name       type   description                                                                  
                      

Author(s)                                                                                                                                  
                     Lena Jaskov         helena.jaskov@uni.lu                                                                               
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         network_analysis                                                                                          
                     Labels       package: kiara_plugin.network_analysis                                                                    
                     References   source_repo: ]8;id=813188;https://github.com/DHARPA-Project/kiara_plugin.network_analysis\https://github.com/DHARPA-Project/kiara_plugin.network_analysis]8;;\                              
                                  documentation: ]8;id=110383;https://DHARPA-Project.github.io/kiara_plugin.network_analysis/\https://DHARPA-Project.github.io/kiara_plugin.network_analysis/]8;;\                            
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Export network data as graphml file.                                                                   
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name        type           description                         Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       network_data      network_data   A value of type 'network_data'.     yes        -- no default --     
                                       base_path         string         The directory to export the         no         -- no default --     
                                                                        file(s) to.                                                         
                                       name              string         The (base) name of the exported     no         -- no default --     
                                                                        file(s).                                                            
                                       export_metadata   boolean        Whether to also export the value    no         False                
                                                                        metadata.                                                           
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name       type   description                                                                  
                      